In [42]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans

In [43]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sat, 23 May 2020 23:02:52 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1321.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Sat, 23 May 2020 19:32:34 GMT', 'Content-Encoding': 'gzip', 'Age': '49116', 'X-Cache': 'cp5008 miss, cp5008 hit/41', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=24-May-2020;Path=/;HttpOnly;secure;Expires=Thu, 25 Jun 2020 12:00:00 GMT, WMF-Last-Access-Global=24-May-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Thu, 25 Jun 2020 12:00:00 GMT, GeoIP=IN:DL:New_Delhi:28.63:77.22:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '182.6

In [44]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [45]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(103, 3)


In [46]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [47]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [49]:
df.shape

(103, 3)

In [50]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [51]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [52]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [53]:
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [54]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [55]:
CLIENT_ID = '35YDWF4I0UGYMISWDWSAF2DWCSBS5GWG23OP3IOXR3WJ4JL2' # Foursquare ID
CLIENT_SECRET = 'LXM1WZFYOE1FS1FRZ3RDO0ZSUBEHYIDDJGQJ2ERQ0OOPWHSN' # Foursquare Secret
VERSION = '20180605' # API version

In [56]:

df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [57]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=35YDWF4I0UGYMISWDWSAF2DWCSBS5GWG23OP3IOXR3WJ4JL2&client_secret=LXM1WZFYOE1FS1FRZ3RDO0ZSUBEHYIDDJGQJ2ERQ0OOPWHSN&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [58]:
results = requests.get(url).json()
# results

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [60]:
venues = results['response']['groups'][0]['items']

In [61]:
nearby_venues = pd.json_normalize(venues) # flatten JSON
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4c633939e1621b8d48842553-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
1,e-0-4be58dc4cf200f479154133c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
2,e-0-4be70e26cf200f47e334153c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,1530 Albion Rd.,at Kipling Ave. (Albion Centre),43.741202,-79.584545,"[{'label': 'display', 'lat': 43.74120165509377...",370,M9V 1B4,CA,Etobicoke,ON,Canada,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
3,e-0-5112b872e4b0c0a78d7bcd27-3,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
4,e-0-4cd4738cdfb4a1cd4337535c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
5,e-0-4b04a05bf964a520c45522e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
6,e-0-4d8ba6910c4e41bdaaf7667f-6,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
7,e-0-4c1951d6834e2d7f2d3a2a80-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
8,e-0-4ce971bff8653704e40db7c4-8,0,"[{'summary': 'This spot is popular', 'type': '...",4ce971bff8653704e40db7c4,LCBO,Albion Mall,1530 Albion Rd,43.741508,-79.584501,"[{'label': 'display', 'lat': 43.74150752194671...",392,M9V 1B4,CA,Toronto,ON,Canada,"[Albion Mall (1530 Albion Rd), Toronto ON M9V ...","[{'id': '4bf58dd8d48988d186941735', 'name': 'L...",0,[]


In [62]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742645,-79.589643
1,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sunny Foodmart,Grocery Store,43.741840,-79.590561
4,The Beer Store,Beer Store,43.741694,-79.584373


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [66]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

KeyError: 'groups'

In [ ]:
print(venues.shape)
venues.head()

In [ ]:

venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

In [ ]:
onehot.shape

In [ ]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

In [ ]:

num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!